# imports

In [1]:
import sys
import os
import numpy as np
import pandas as pd
# mcfly
from mcfly import modelgen, find_architecture, storage
from keras.models import load_model
np.random.seed(2)

Using TensorFlow backend.


In [2]:
sys.path.insert(0, os.path.abspath('../..'))
from utils import tutorial_vu

# Load data

In [13]:
X_train, y_train, X_val, y_val, X_test, y_test = tutorial_vu.load_one_time_fall_dataset(base_path='/data/Data espen/')

X_train = X_train[:, 1000:1500, :]
X_val = X_val[:, 1000:1500, :]
X_test = X_test[:, 1000:1500, :]

Loading training data.
Processing file Acc_Vel_gait_30sec_77.mat
Processing file Acc_Vel_gait_30sec_10.mat
Processing file Acc_Vel_gait_30sec_74.mat
Processing file Acc_Vel_gait_30sec_153.mat
Processing file Acc_Vel_gait_30sec_58.mat
Processing file Acc_Vel_gait_30sec_127.mat
Processing file Acc_Vel_gait_30sec_89.mat
Processing file Acc_Vel_gait_30sec_97.mat
Processing file Acc_Vel_gait_30sec_91.mat
Processing file Acc_Vel_gait_30sec_185.mat
Processing file Acc_Vel_gait_30sec_29.mat
Processing file Acc_Vel_gait_30sec_155.mat
Processing file Acc_Vel_gait_30sec_130.mat
Processing file Acc_Vel_gait_30sec_191.mat
Processing file Acc_Vel_gait_30sec_31.mat
Processing file Acc_Vel_gait_30sec_147.mat
Processing file Acc_Vel_gait_30sec_78.mat
Processing file Acc_Vel_gait_30sec_160.mat
Processing file Acc_Vel_gait_30sec_22.mat
Processing file Acc_Vel_gait_30sec_183.mat
Processing file Acc_Vel_gait_30sec_15.mat
Processing file Acc_Vel_gait_30sec_123.mat
Processing file Acc_Vel_gait_30sec_70.mat
P

In [22]:
from keras.utils.np_utils import to_categorical
y_train_binary = to_categorical(y_train)
y_val_binary = to_categorical(y_val)
y_test_binary = to_categorical(y_test)

# Inspect data

In [23]:
print('train set size:', X_train.shape[0])
print('validation set size:', X_val.shape[0])
print('test set size:', X_test.shape[0])

train set size: 17691
validation set size: 2582
test set size: 4113


In [24]:
print('x shape:', X_train.shape)
print('y shape:', y_train.shape)

x shape: (17691, 500, 6)
y shape: (17691,)


# Generate models

In [25]:
num_classes = y_train_binary.shape[1]

models = modelgen.generate_models(X_train.shape,
                                  number_of_classes=num_classes,
                                  number_of_models = 5)

Let's inspect the models:

In [26]:
models[0]

(<keras.models.Sequential at 0x7f3be6eaa5f8>,
 {'fc_hidden_nodes': 1014,
  'filters': array([35, 98, 90, 23, 67, 53, 82]),
  'learning_rate': 0.0001112405218646149,
  'regularization_rate': 0.014227883110424682},
 'CNN')

In [27]:
models[0][0].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_25 (Batc (None, 500, 6)            24        
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 500, 35)           665       
_________________________________________________________________
batch_normalization_26 (Batc (None, 500, 35)           140       
_________________________________________________________________
activation_25 (Activation)   (None, 500, 35)           0         
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 500, 98)           10388     
_________________________________________________________________
batch_normalization_27 (Batc (None, 500, 98)           392       
_________________________________________________________________
activation_26 (Activation)   (None, 500, 98)           0         
__________

In [ ]:

for i, item in enumerate(models):
    model, params, model_types = item
    print("-------------------------------------------------------------------------------------------------------")
    print("Model " + str(i))
    print(" ")
    print("Hyperparameters:")
    print(params)
    print(" ")
    print("Model description:")
    model.summary()
    print(" ")
    print("Model type:")
    print(model_types)
    print(" ")

-------------------------------------------------------------------------------------------------------
Model 0
 
Hyperparameters:
{'learning_rate': 0.0001112405218646149, 'regularization_rate': 0.014227883110424682, 'filters': array([35, 98, 90, 23, 67, 53, 82]), 'fc_hidden_nodes': 1014}
 
Model description:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_25 (Batc (None, 500, 6)            24        
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 500, 35)           665       
_________________________________________________________________
batch_normalization_26 (Batc (None, 500, 35)           140       
_________________________________________________________________
activation_25 (Activation)   (None, 500, 35)           0         
_________________________________________________________________
conv1d_15 (Conv1D)           

# Compare the models

In [ ]:
outputfile = 'modelcomparison.json'
histories, val_accuracies, val_losses = find_architecture.train_models_on_samples(X_train, y_train_binary,
                                                                           X_val[:100], y_val_binary[:100],
                                                                           models,nr_epochs=5,
                                                                           subset_size=1000,
                                                                           verbose=True,
                                                                           outputfile=outputfile,
                                                                           batch_size=5)
print('Details of the training process were stored in ',outputfile)

Training model 0 CNN
Train on 1000 samples, validate on 100 samples
Epoch 1/5
1000/1000 [==============================] - 35s - loss: 14.3085 - acc: 0.5010 - val_loss: 9.8883 - val_acc: 0.5100
Epoch 2/5
1000/1000 [==============================] - 31s - loss: 8.2035 - acc: 0.6070 - val_loss: 7.1187 - val_acc: 0.4900
Epoch 3/5
1000/1000 [==============================] - 31s - loss: 6.4838 - acc: 0.6470 - val_loss: 6.1279 - val_acc: 0.5300
Epoch 4/5
1000/1000 [==============================] - 31s - loss: 5.8758 - acc: 0.6680 - val_loss: 6.1135 - val_acc: 0.3900
Epoch 5/5
1000/1000 [==============================] - 31s - loss: 5.4427 - acc: 0.7560 - val_loss: 5.8046 - val_acc: 0.4800
Training model 1 CNN
Train on 1000 samples, validate on 100 samples
Epoch 1/5
1000/1000 [==============================] - 15s - loss: 19.3306 - acc: 0.5220 - val_loss: 6.1197 - val_acc: 0.4000
Epoch 2/5
1000/1000 [==============================] - 13s - loss: 3.2597 - acc: 0.5400 - val_loss: 2.0461 - val